### ⚠ IMPORTANT ⚠

You will need at least 22GB of VRAM (GPU RAM) to run this notebook.

If you're running this locally - please ensure you have the correct hardware to support the fine-tuning.

Please make sure you're using the following instance:

![image](https://i.imgur.com/ji210Ug.png)

# Fine-tuning Embedding Models

In the following Notebook we will be exploring one of the most powerful techniques to take your single-domain RAG pipelines to the next level.

Fine-tuning Embeddings Models!

- 🤝 Breakout Room #2
  - Task 1: Dependencies and Boilerplate
  - Task 2: Loading Data
  - Task 3: Constructing a Fine-tuning Dataset
  - Task 4: Fine-tuning `snowflake-arctic-embed-l`
  - Task 5: Evaluating Retrieval with Embedding Model

But before any of that, we need to grab some dependencies, and set up some boilerplate!

## Task 1: Dependencies and Boilerplate

We'll set up our `nest_asyncio` so we can leverage async loops in our Notebook.

We'll also install the required libraries we'll be using today, and set up our OpenAI API key, and Hugging Face token!

### Nest Asyncio

In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Install Dependencies

In [ ]:
!pip install -qU llama-index-llms-openai llama-index-embeddings-openai llama-index-finetuning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.5/375.5 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 104.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
!pip install -qU llama-index-readers-file llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 13.8 MB/s eta 0:00:00


In [ ]:
!pip install -qU "sentence_transformers==2.7.0"

### API Key Section!

In classic fashion, we'll need to provide our OpenAI API key!

We'll also provide our Hugging Face token (with `Write` access) in order to save our model on the Hub!

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

Enter Your OpenAI API Key: ··········


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Task 2: Loading Data

The data can be found in [this GitHub repo](https://github.com/AI-Maker-Space/DataRepository/tree/main/high-performance-rag).

In this case, the data is related to research articles about Camelids (aka: Llamas, Alpacas, Camels!)

In [ ]:
!git clone https://github.com/AI-Maker-Space/DataRepository.git

Cloning into 'DataRepository'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 71 (delta 19), reused 28 (delta 8), pack-reused 8
Receiving objects: 100% (71/71), 69.00 MiB | 13.97 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [ ]:
%cd ./DataRepository/high-performance-rag/

/content/DataRepository/high-performance-rag


In [ ]:
!unzip "Camel Papers Test.zip"

Archive:  Camel Papers Test.zip
  inflating: Camel Papers Test/Acute respiratory distress syndrome in an alpaca cria.pdf  
  inflating: Camel Papers Test/Alpaca liveweight variations and fiber production in Mediterranean range of Chile.pdf  


In [ ]:
!unzip "Camel Papers Train.zip"

Archive:  Camel Papers Train.zip
  inflating: Camel Papers Train/Antibody response to the epsilon toxin ofClostridium perfringensfollowing vaccination of Lama glamacrias.pdf  
  inflating: Camel Papers Train/Comparative pigmentation of sheep, goats, and llamas what colors are possible through selection.pdf  
  inflating: Camel Papers Train/Conservative management of a ruptured.pdf  
  inflating: Camel Papers Train/Evaluation of cholesterol and vitamin E concentrations in adult alpacas and nursing crias.pdf  
  inflating: Camel Papers Train/Influence of effects on quality traits and relationships between traits of the llama fleece..pdf  
  inflating: Camel Papers Train/Influence of Follicular Fluid on in Vitro.pdf  
  inflating: Camel Papers Train/Neurological Causes of Diaphragmatic Paralysis in 11 Alpacas.pdf  
  inflating: Camel Papers Train/On the morphology of the cerebellum of the alpaca (Lama pacos)..pdf  
  inflating: Camel Papers Train/Relationships between integumental charact

Now we can begin building our simple index for each of the training directories, and the validation directories.

We will use LlamaIndex's `SimpleNodeParser` to achieve this!

In [ ]:
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import MetadataMode

TRAIN_FILES = "Camel Papers Train"
EVAL_FILES = "Camel Papers Test"

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import MetadataMode

def load_corpus(directory, verbose=False):
    if verbose:
        print(f"Loading files in {directory}")

    reader = SimpleDirectoryReader(directory)
    docs = reader.load_data()
    if verbose:
        print(f"Loaded {len(docs)} docs")

    parser = SimpleNodeParser.from_defaults()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f"Parsed {len(nodes)} nodes")

    return nodes

In [ ]:
train_nodes = load_corpus(TRAIN_FILES, verbose=True)
eval_nodes = load_corpus(EVAL_FILES, verbose=True)

Loading files in Camel Papers Train
Loaded 91 docs


Parsing nodes:   0%|          | 0/91 [00:00<?, ?it/s]

Parsed 156 nodes
Loading files in Camel Papers Test
Loaded 9 docs


Parsing nodes:   0%|          | 0/9 [00:00<?, ?it/s]

Parsed 17 nodes


Now that we've split our source documents into a number of nodes, we can move on to constructing a fine-tuning dataset.

## Task 3: Constructing a Fine-tuning Dataset

Using the nodes we created above, we can finally start constructing a fine-tuning dataset utilizing OpenAI's `gpt-3.5-turbo`.

We'll start by using LlamaIndex's `generate_qa_embedding_pairs` and storing it in a `EmbeddingQAFinetuneDataset`.

The basic idea here is straightforward enough:

1. We look at a node
2. We generate a question that could be answered by that node

This gives us a number of question/context pairs that we can use to fine-tune our Embeddings model.

> NOTE: Keep in mind that the below example uses 100 nodes to generate the QA pairs. This results in 100 calls to `gpt-3.5-turbo` feel free to reduce the number of nodes.

In [ ]:
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(temperature=0.0, model="gpt-3.5-turbo")

In [ ]:
train_dataset = generate_qa_embedding_pairs(train_nodes[:100], llm=llm)
train_dataset.save_json("train_dataset.json")

100%|██████████| 100/100 [02:51<00:00,  1.72s/it]


In [ ]:
eval_dataset = generate_qa_embedding_pairs(eval_nodes[:10], llm=llm)
eval_dataset.save_json("eval_dataset.json")

100%|██████████| 10/10 [00:15<00:00,  1.51s/it]


In [ ]:
train_dataset = EmbeddingQAFinetuneDataset.from_json("train_dataset.json")
eval_dataset = EmbeddingQAFinetuneDataset.from_json("eval_dataset.json")

## Task 4: Fine-tuning `snowflake-arctic-embed-l`

Now that we have a dataset, let's grab a `sentence-transformers` Embeddings model!

We'll be using Snowflake's [`snowflake-arctic-embed-l`](https://huggingface.co/Snowflake/snowflake-arctic-embed-l) as a base embeddings model.

It is a well performing embeddings model by itself, but there's a lot of very specific domain terms and vocabulary in our courpus - so lets fine-tune it and see what that can do for us!

> NOTE: If you are limited by your compute - you can use the `snowflake-arctic-embed-m` model instead, which will run on the free T4 GPU instance in Colab.

####❓ Question 1:

How many parameters does `snowflake-arctic-embed-l` have?

snowflake-arctic-embed-l:
- 334 Million parameters
- 1024 Dimensions
- 512 Context windows

In [ ]:
from llama_index.finetuning import SentenceTransformersFinetuneEngine

finetune_engine = SentenceTransformersFinetuneEngine(
    train_dataset, # Dataset to be trained on
    val_dataset=eval_dataset, # Dataset to evaluate on
    model_id="Snowflake/snowflake-arctic-embed-l", # HuggingFace reference to base embeddings model
    model_output_path="snowflake_finetune_camelids", # Output directory for fine-tuned embeddings model
    epochs=4 # Number of Epochs to train for
)

All that's left to do now is call `.finetune()`!

In [ ]:
finetune_engine.finetune()

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Now that we've fine-tuned our embeddings model, lets grab the model out of the engine so we can use it later!

> NOTE: You should be able to safely avoid any warnings relating to weights here.

In [ ]:
finetuned_embedding_model = finetune_engine.get_finetuned_model()




Some weights of BertModel were not initialized from the model checkpoint at snowflake_finetune_camelids and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sentence_transformers import SentenceTransformer

fine_tuned_embedding = SentenceTransformer(
    "snowflake_finetune_camelids"
)




Some weights of BertModel were not initialized from the model checkpoint at snowflake_finetune_camelids and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
fine_tuned_embedding.save_to_hub(repo_id="MiguelCosta/snowflake-ft-camelids-l-4epoch")

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

'https://huggingface.co/MiguelCosta/snowflake-ft-camelids-l-4epoch/commit/610daf0d750524d730c0759f9d8058294c31adfa'

## Task 5: Evaluating Retrieval with Embedding Model

Now that we've fine-tuned our model - let's see how it performs against OpenAI's `text-embedding-3-small` model, and the base non-fine-tuned version of the model.

In [ ]:
from tqdm.notebook import tqdm
from llama_index.core.schema import TextNode
from llama_index.core import Settings, VectorStoreIndex


def evaluate(
    dataset,
    embed_model,
    top_k=2,
    verbose=False,
):
    corpus = dataset['corpus']
    queries = dataset['queries']
    relevant_docs = dataset['relevant_docs']

    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items() if text != ""]
    index = VectorStoreIndex(
        nodes,
        show_progress=True,
        embed_model=embed_model
    )
    retriever = index.as_retriever(similarity_top_k=top_k)

    eval_results = []
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]
        is_hit = expected_id in retrieved_ids  # assume 1 relevant doc

        eval_result = {
            'is_hit': is_hit,
            'retrieved': retrieved_ids,
            'expected': expected_id,
            'query': query_id,
        }
        eval_results.append(eval_result)
    return eval_results

####❓Question 2:

Describe what the `evaluate` function is doing in the above cell in natural language.

it checks for each entry in the dataset if ANY of the retrieved ids, (in our case =2) matches the ground truth given by the generator model (gpt-3.5-turbo)
The result is the mean of all the matches.

In [ ]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer

def evaluate_sentence_transformers(
    dataset,
    model_id,
    name,
):
    corpus = dataset['corpus']
    queries = dataset['queries']
    relevant_docs = dataset['relevant_docs']

    evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs, name=name)
    model = SentenceTransformer(model_id)
    return evaluator(model, output_path="/content/")

####❓Question 3:

Describe what the `evaluate_st` function is doing in the above cell in natural language.

It does the same as the function "evaluate", but with a variation of the metrics. 
"Given a set of queries and a large corpus set. It will retrieve for each query the top-k most similar document.
It measures Mean Reciprocal Rank (MRR), Recall@k, and Normalized Discounted Cumulative Gain (NDCG)

In [ ]:
import json

with open("eval_dataset.json", 'r+') as f:
    eval_dataset_json = json.load(f)

### Text Embedding 3 Small Results

We'll compare our results against OpenAI's `text-embedding-3-small` model, so we'll need to load it up!

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding

text_embedding_3_small = OpenAIEmbedding(model="text-embedding-3-small")
te3_val_results = evaluate(eval_dataset_json, text_embedding_3_small)

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Let's look at what an example of our results looks like.

In [ ]:
import pandas as pd

df_te3 = pd.DataFrame(te3_val_results)

In [ ]:
df_te3

,is_hit,retrieved,expected,query
0,True,"[9b0adbef-ab3c-4a24-9c98-43c8b270034b, 29336dd...",9b0adbef-ab3c-4a24-9c98-43c8b270034b,9c84a317-ff85-4c5f-889a-71d599f4c5cc
1,True,"[9b0adbef-ab3c-4a24-9c98-43c8b270034b, 468e081...",9b0adbef-ab3c-4a24-9c98-43c8b270034b,3a94c729-849b-4d0c-b2bd-70b70a4bd64c
2,True,"[cb1df031-8518-417f-8d4e-df528ca4b52a, 70f8070...",cb1df031-8518-417f-8d4e-df528ca4b52a,9749690e-5e07-480c-aeb5-a906edff853e
3,True,"[cb1df031-8518-417f-8d4e-df528ca4b52a, 468e081...",cb1df031-8518-417f-8d4e-df528ca4b52a,b8b3f0dc-d201-4aa8-96d7-ea9e1a3847f8
4,True,"[fff9b051-2006-4382-8f77-b3d033d720e5, 468e081...",fff9b051-2006-4382-8f77-b3d033d720e5,0528d04f-95c4-424c-aedb-9194555623d8
5,True,"[468e081f-5ded-496e-af60-a8e97c1c906d, fff9b05...",fff9b051-2006-4382-8f77-b3d033d720e5,c0af7d17-cafe-49d2-aa3b-46424d4d52f6
6,True,"[468e081f-5ded-496e-af60-a8e97c1c906d, 29336dd...",468e081f-5ded-496e-af60-a8e97c1c906d,bcfc9e9a-3373-4763-9cfc-da94057c8170
7,True,"[468e081f-5ded-496e-af60-a8e97c1c906d, 29336dd...",468e081f-5ded-496e-af60-a8e97c1c906d,821dfe21-9d9f-4b88-b3ba-aae65b28d6b3
8,False,"[70f8070d-d9b1-450e-b4fd-cc131fd149a0, a16b40f...",29336ddf-d4d3-4502-af92-cc10129df00b,0c84a939-ddc9-418c-b1c7-61469063605b
9,True,"[29336ddf-d4d3-4502-af92-cc10129df00b, 70f8070...",29336ddf-d4d3-4502-af92-cc10129df00b,7efa48c1-f9b5-4349-9c0a-eaef0a509db2


####❓Question 4:

What do these `[313de41e-534b...]` IDs mean?

These are the node ids (for each chunk).

Now let's look at the mean value of `is_hit`.

In [ ]:
hit_rate_ada = df_te3['is_hit'].mean()
hit_rate_ada

0.95

Overall, we see `text-embedding-3-small` getting a `0.9` "hit rate".

### Base Embeddings Model Results

Let's get the evaluation for our base embedding model (pre-fine-tuning).

In [ ]:
base_embed_model_id = "Snowflake/snowflake-arctic-embed-l"
base_embed_model = SentenceTransformer(base_embed_model_id)

arctic_base = "local:Snowflake/snowflake-arctic-embed-l"
arctic_base_val_results = evaluate(eval_dataset_json, arctic_base)

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
df_arctic_base = pd.DataFrame(arctic_base_val_results)

In [ ]:
hit_rate_bge = df_arctic_base['is_hit'].mean()
hit_rate_bge

0.5

With a `0.5` hit rate - the base embedding model is absolutely terrible when compared to `text-embedding-3-small` from OpenAI!

Because this is a local `SentenceTransformer`, we can evaluate it with the `SentenceTransformer` evaluation helper-function as well!

In [ ]:
evaluate_sentence_transformers(eval_dataset_json, "Snowflake/snowflake-arctic-embed-l", name='arctic-l')

0.5271428571428571

Not great results - let's see what fine-tuning can do for us!

### Fine-tuned Results

In [ ]:
finetuned = "local:snowflake_finetune_camelids"
eval_results_finetuned = evaluate(eval_dataset_json, finetuned)




Some weights of BertModel were not initialized from the model checkpoint at snowflake_finetune_camelids and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
df_finetuned = pd.DataFrame(eval_results_finetuned)

In [ ]:
hit_rate_finetuned = df_finetuned['is_hit'].mean()
hit_rate_finetuned

0.8

This is a marked improvement when compared to the base model. Absolutely fantastic!

In [ ]:
evaluate_sentence_transformers(eval_dataset_json, "snowflake_finetune_camelids", name='finetuned')




Some weights of BertModel were not initialized from the model checkpoint at snowflake_finetune_camelids and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.8291666666666666

It's also a marked improvement on the `SentenceTransformer` evaluation!

### Conclusion

Now we can compare the 3 embeddings models to see which performed the best!

In [ ]:
df_te3['model'] = 'te3'
df_arctic_base['model'] = 'arctic-baseline'
df_finetuned['model'] = 'arctic-fine-tuned'

In [ ]:
df_all = pd.concat([df_te3, df_arctic_base, df_finetuned])
df_all.groupby('model').mean('is_hit')

,is_hit
model,
arctic-baseline,0.50
arctic-fine-tuned,0.85
te3,0.95


The cell above runned with the model version trainned for 10 epochs
(https://huggingface.co/MiguelCosta/snowflake-ft-camelids-l)

The cell below runned with the model version trainned for 4 epochs


In [ ]:
df_all = pd.concat([df_te3, df_arctic_base, df_finetuned])
df_all.groupby('model').mean('is_hit')

,is_hit
model,
arctic-baseline,0.50
arctic-fine-tuned,0.80
te3,0.95
